<a href="https://colab.research.google.com/github/tejaschaudhari2811/FIRE_Project_Task_1/blob/main/FIRE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1 : Message Level Sentiment Analysis for Tamil Language

In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import re
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
print("Upload Training Data \n")
from google.colab import files
uploaded = files.upload()

Upload Training Data 



Saving tamil_offensive_train.tsv to tamil_offensive_train.tsv


In [ ]:
df = pd.read_table("tamil_offensive_train.tsv")

In [ ]:
# Define Polarities for numeric classification
polarities={'NOT':1, "OFF":-1,"not-Tamil":0}

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5880 entries, 0 to 5879
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5880 non-null   object
 1   text      5880 non-null   object
 2   category  5880 non-null   object
dtypes: object(3)
memory usage: 137.9+ KB
None


In [ ]:
df.groupby(by=['category']).count()

,id,text
category,,
NOT,4724,4724
OFF,1153,1153
not-Tamil,3,3


In [ ]:
# Supplementary Functions
def convert_polarity(category):
    return polarities[category]

def word_length(sentence):
    return len(sentence.split(" "))

In [ ]:
df['polarity'] = df['category'].apply(convert_polarity)
df['text_length'] = df['text'].apply(len)
df['number_of_words'] = df['text'].apply(word_length)

In [ ]:
df.head()

,id,text,category,polarity,text_length,number_of_words
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT,1,60,8
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF,-1,80,12
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT,1,69,7
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF,-1,84,13
4,tam5,only for விஜய் சேதுபதி and STR,NOT,1,30,6


In [ ]:
df = df.drop(df[df.polarity == 0].index)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
tfidf = TfidfVectorizer(max_features=1000)

In [ ]:
# Get the data from Dataframe.
X = df['text']
y = df['polarity']

# Convert data to vectors
X = tfidf.fit_transform(X)
X_ros,y_ros = ros.fit_resample(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def svm_classifier(X_ros,y_ros):
  X_train,X_test,y_train,y_test = train_test_split(X_ros,y_ros,test_size=0.2, random_state=0)
  clf = LinearSVC()
  linear_svc_model = clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  report = classification_report(y_pred,y_test)
  return linear_svc_model,report


In [ ]:
model,report= svm_classifier(X_ros,y_ros)

In [ ]:
print(report)

              precision    recall  f1-score   support

          -1       0.85      0.78      0.81      1041
           1       0.75      0.83      0.79       849

    accuracy                           0.80      1890
   macro avg       0.80      0.81      0.80      1890
weighted avg       0.81      0.80      0.80      1890



# Using Logistic Regression for Classification

In [ ]:
def logistic_regression_classifier(X_ros,y_ros):
  X_train,X_test,y_train,y_test = train_test_split(X_ros,y_ros,test_size=0.2, random_state=0)
  clf = LogisticRegression(random_state=0)
  logistic_regression_model = clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  report = classification_report(y_pred,y_test)
  return logistic_regression_model,report

In [ ]:
lr_model,report = logistic_regression_classifier(X_ros,y_ros)
print(report)

              precision    recall  f1-score   support

          -1       0.81      0.78      0.80       995
           1       0.77      0.80      0.78       895

    accuracy                           0.79      1890
   macro avg       0.79      0.79      0.79      1890
weighted avg       0.79      0.79      0.79      1890



# Use of Indic-nlp-library for Text Processing

In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_resources.git"

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 31.48 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (28/28), done.


In [ ]:
!pip install indic-nlp-library

     |████████████████████████████████| 40kB 4.9MB/s 
     |████████████████████████████████| 9.2MB 29.7MB/s 
     |████████████████████████████████| 552kB 41.5MB/s 
  Created wheel for sphinx-argparse: filename=sphinx_argparse-0.2.5-cp37-none-any.whl size=11552 sha256=24ae4d693fc3c3aa94e3a2b0be2ef52b8d417075acddf04f51d36cad00d9f842
  Stored in directory: /root/.cache/pip/wheels/2a/18/1b/4990a1859da4edc77ab312bc2986c08d2733fb5713d06e44f5
Successfully built sphinx-argparse
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1


In [ ]:
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"

In [ ]:
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
from indicnlp import loader
loader.load()

In [ ]:
from indicnlp.tokenize import sentence_tokenize, indic_tokenize
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator, ItransTransliterator
string = "திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்"
sentences = sentence_tokenize.sentence_split(string, lang='ta')
for se in indic_tokenize.trivial_tokenize(string):
  print(ItransTransliterator.to_itrans(se,'hi'))

திருமலை
நாயக்கர்
பேரவை
சார்பாக
படம்
வெற்றி
பெற
வாழ்த்துக்கள்


# Using Huggingface Features


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 12.7MB/s 
     |████████████████████████████████| 901kB 53.8MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 


In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained("bert-base-multilingual-uncased")


Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
text = "திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

In [ ]:
X_hf = df['text']
y_hf = df['polarity']

X_hf = X_hf.apply(tokenizer)

In [ ]:
X_hf[0]

{'input_ids': [101, 54552, 60211, 25088, 808, 37025, 20798, 13047, 810, 55541, 20841, 802, 15241, 85286, 12146, 810, 44042, 818, 96902, 49815, 810, 96902, 36718, 818, 13744, 40806, 95588, 12076, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
import numpy as np

In [ ]:
# Remove Unnecessary keys and values from the embeddings
X_train = []
for text in X_hf:
  text.pop('token_type_ids')
  text.pop('attention_mask')
  X_train.append(np.array(text['input_ids']))

print("Finished")

Finished


In [ ]:
X_ros,y_ros = ros.fit_resample(X_train,y_hf)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: ignored